In [2]:
import os
import numpy as np
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("CUB_200_2011/images.txt", sep="/n")  

C:\Users\Richard Patrick\AppData\Local\Temp\ipykernel_23712\3715118202.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("CUB_200_2011/images.txt", sep="/n")


In [4]:
df

,1 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
0,2 001.Black_footed_Albatross/Black_Footed_Alba...
1,3 001.Black_footed_Albatross/Black_Footed_Alba...
2,4 001.Black_footed_Albatross/Black_Footed_Alba...
3,5 001.Black_footed_Albatross/Black_Footed_Alba...
4,6 001.Black_footed_Albatross/Black_Footed_Alba...
...,...
11782,11784 200.Common_Yellowthroat/Common_Yellowthr...
11783,11785 200.Common_Yellowthroat/Common_Yellowthr...
11784,11786 200.Common_Yellowthroat/Common_Yellowthr...
11785,11787 200.Common_Yellowthroat/Common_Yellowthr...
